In [2]:
import os 
import math
import torch
import tqdm
import copy
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
print("Done!")

Done!


In [3]:
# Load of all the .csv files containing extracted image features
data_USG = pd.read_csv("Features/Ultrasound features.csv")
data_MMG = pd.read_csv("Features/Mammogram features.csv")
data_multimodal = pd.read_csv("Features/multimodal features.csv")
print('Done!')

Done!


In [4]:
#For one hot encoding the labels this tab is required
class_mapping = {
    'B': 0,
    'M': 1,
}
num_classes = 2

label_encoder = LabelEncoder()
data_USG['Class'] = label_encoder.fit_transform(data_USG['Class'])
data_MMG['Class'] = label_encoder.fit_transform(data_MMG['Class'])
data_multimodal['Class'] = label_encoder.fit_transform(data_multimodal['Class'])
print('Done!')

Done!


TCBlock retains information and details(weights) from the bool of information(Temporal Convolutions) it is not being used as there is no sequential data in our files

In [4]:
# #Define TCBlock 
# class TCBlock(nn.Module):
#     def __init__(self, in_channels, seq_length, filters):
#         super(TCBlock, self).__init__()
#         self.dense_blocks = nn.ModuleList([DenseBlock(in_channels + i * filters, 2 ** (i+1), filters)
#                                            for i in range(int(math.ceil(math.log(seq_length, 2))))])

#     def forward(self, input):
#         # input is dimensions (N, T, in_channels)
#         input = torch.transpose(input, 1, 2)
#         for block in self.dense_blocks:
#             input = block(input)
#         return torch.transpose(input, 1, 2)

Attention Block pin points information in a seemingly large set of data and retains it to pass on to the temporal Convolution block

In [5]:
class AttentionBlock(nn.Module):
    def __init__(self, in_channels, key_size, value_size):
        super(AttentionBlock, self).__init__()
        self.linear_query = nn.Linear(in_channels, key_size)
        self.linear_keys = nn.Linear(in_channels, key_size)
        self.linear_values = nn.Linear(in_channels, value_size)
        self.sqrt_key_size = math.sqrt(key_size)

    def forward(self, input):
        # input is dim (N, T, in_channels) where N is the batch_size, and T is
        # the sequence length
        batch_size, seq_length, in_channels = input.size()
        
        # Create an upper triangular mask
        mask = torch.triu(torch.ones(seq_length, seq_length, device=input.device), diagonal=1).byte()
        
        keys = self.linear_keys(input)  # shape: (N, T, key_size)
        query = self.linear_query(input)  # shape: (N, T, key_size)
        values = self.linear_values(input)  # shape: (N, T, value_size)
        
        temp = torch.bmm(query, keys.transpose(1, 2))  # shape: (N, T, T)
        temp.data.masked_fill_(mask, -float('inf'))
        
        temp = F.softmax(temp / self.sqrt_key_size, dim=1)  # shape: (N, T, T)
        temp = torch.bmm(temp, values)  # shape: (N, T, value_size)
        
        return torch.cat((input, temp), dim=2)  # shape: (N, T, in_channels + value_size)

print('Done!')

Done!


Define the SNAIL architecture 

In [6]:
class SnailFewShot(nn.Module):
    def __init__(self, input_dim):
        super(SnailFewShot, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),  # Adjust the hidden layer size as needed
            nn.ReLU(),
            nn.Linear(64, 32),  # You can adjust the hidden layer size here as well
            nn.ReLU()
        )
        self.attention = AttentionBlock(32, 8, 8)  # Adjust key_size and value_size as needed
        self.fc = nn.Linear(32 + 8, 1)  # Output a single value for binary classification

    def forward(self, x):
        x = self.encoder(x)
        x = self.attention(x)
        x = self.fc(x)
        return torch.sigmoid(x)  # Sigmoid activation for binary classification

# Create instances of the SnailFewShot model for both input shapes
input_dim_512 = 512  # Number of features in the (512, 0) input
model_512 = SnailFewShot(input_dim_512)

input_dim_1024 = 1024  # Number of features in the (1024, 0) input
model_1024 = SnailFewShot(input_dim_1024)

print("Done!")

Done!


From this point on the data is run and the main block is executed

In [9]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler

# # ... (Data Preparation)

# # Initialize the model
# input_dim = X_train.shape[1]  # Number of features
# model = SnailFewShot(input_dim)

# # Define loss and optimizer
# criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
# optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate as needed

# # Training loop
# num_epochs = 10  # Adjust as needed
# for epoch in range(num_epochs):
#     model.train()
#     total_loss = 0.0
#     for batch_X, batch_y in train_loader:
#         optimizer.zero_grad()
#         predictions = model(batch_X)
#         loss = criterion(predictions, batch_y.view(-1, 1))  # Ensure y has shape (batch_size, 1)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

# # Evaluation
# model.eval()
# with torch.no_grad():
#     correct = 0
#     total = 0
#     for batch_X, batch_y in test_loader:
#         predictions = model(batch_X)
#         predicted_labels = (predictions >= 0.5).float()
#         total += batch_y.size(0)
#         correct += (predicted_labels == batch_y.view(-1, 1)).sum().item()  # Ensure y has shape (batch_size, 1)

#     accuracy = 100 * correct / total
#     print(f"Test Accuracy: {accuracy:.2f}%")


In [8]:
# Extract features and labels
X_USG = data_USG.drop(columns=['Class']).values
y_USG = data_USG['Class'].values  # Assuming labels are 0 or 1 for binary classification

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_USG, y_USG, test_size=0.2, random_state=42)

# Standardize features (optional but often recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # Reshape for binary classification
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)  # Reshape for binary classification

# Create DataLoader for training and test sets
batch_size = 32  # Adjust as needed
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Initialize the model
input_dim = X_train.shape[1]  # Number of features
model = SnailFewShot(input_dim)

# Define loss and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate as needed

# Training loop
num_epochs = 10  # Adjust as needed
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        predictions = model(batch_X)
        loss = criterion(predictions, batch_y.view(-1, 1))  # Ensure y has shape (batch_size, 1)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch_X, batch_y in test_loader:
        predictions = model(batch_X)
        predicted_labels = (predictions >= 0.5).float()
        total += batch_y.size(0)
        correct += (predicted_labels == batch_y.view(-1, 1)).sum().item()  # Ensure y has shape (batch_size, 1)

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")


ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
# Evaluate the model and get predictions
model.eval()
model_predictions = []

with torch.no_grad():
    for batch_X, _ in test_loader:
        predictions = model(batch_X)
        model_predictions.extend(predictions.cpu().numpy())

# Convert predicted probabilities to binary labels (0 or 1)
predicted_labels = (np.array(model_predictions) >= 0.5).astype(int)

# Convert true labels to a numpy array
true_labels = y_test_tensor.cpu().numpy().astype(int)

# Create a confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, xticklabels=["Benign", "Malignant"], yticklabels=["Benign", "Malignant"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Extract features and labels
X_MMG = data_MMG.drop(columns=['Class']).values
y_MMG = data_MMG['Class'].values  # Assuming labels are 0 or 1 for binary classification

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_MMG, y_MMG, test_size=0.2, random_state=42)

# Standardize features (optional but often recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # Reshape for binary classification
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)  # Reshape for binary classification

# Create DataLoader for training and test sets
batch_size = 32  # Adjust as needed
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Initialize the model
input_dim = X_train.shape[1]  # Number of features
model = SnailFewShot(input_dim)

# Define loss and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate as needed

# Training loop
num_epochs = 10  # Adjust as needed
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        predictions = model(batch_X)
        loss = criterion(predictions, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch_X, batch_y in test_loader:
        predictions = model(batch_X)
        predicted_labels = (predictions >= 0.5).float()
        total += batch_y.size(0)
        correct += (predicted_labels == batch_y).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")


In [ ]:
# Evaluate the model and get predictions
model.eval()
model_predictions = []

with torch.no_grad():
    for batch_X, _ in test_loader:
        predictions = model(batch_X)
        model_predictions.extend(predictions.cpu().numpy())

# Convert predicted probabilities to binary labels (0 or 1)
predicted_labels = (np.array(model_predictions) >= 0.5).astype(int)

# Convert true labels to a numpy array
true_labels = y_test_tensor.cpu().numpy().astype(int)

# Create a confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, xticklabels=["Benign", "Malignant"], yticklabels=["Benign", "Malignant"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Extract features and labels
X = data_multimodal.drop(columns=['Class']).values
y = data_multimodal['Class'].values  # Assuming labels are 0 or 1 for binary classification

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features (optional but often recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # Reshape for binary classification
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)  # Reshape for binary classification

# Create DataLoader for training and test sets
batch_size = 64  # Adjust as needed
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Initialize the model
input_dim = X_train.shape[1]  # Number of features
model = SnailFewShot(input_dim)

# Define loss and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate as needed

# Training loop
num_epochs = 20  # Adjust as needed
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        predictions = model(batch_X)
        loss = criterion(predictions, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch_X, batch_y in test_loader:
        predictions = model(batch_X)
        predicted_labels = (predictions >= 0.5).float()
        total += batch_y.size(0)
        correct += (predicted_labels == batch_y).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")


In [ ]:
# Evaluate the model and get predictions
model.eval()
model_predictions = []

with torch.no_grad():
    for batch_X, _ in test_loader:
        predictions = model(batch_X)
        model_predictions.extend(predictions.cpu().numpy())

# Convert predicted probabilities to binary labels (0 or 1)
predicted_labels = (np.array(model_predictions) >= 0.5).astype(int)

# Convert true labels to a numpy array
true_labels = y_test_tensor.cpu().numpy().astype(int)

# Create a confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, xticklabels=["Class 0", "Class 1"], yticklabels=["Class 0", "Class 1"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()